In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Imagen Product Recontextualization: Image Generation


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/imagen_product_recontext.ipynb">
      <img src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fvision%2Fgetting-started%2Fimagen_product_recontext.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/vision/getting-started/imagen_product_recontext.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/imagen_product_recontext.ipynb">
      <img width="32px" src="https://raw.githubusercontent.com/primer/octicons/refs/heads/main/icons/mark-github-24.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/imagen_product_recontext.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/imagen_product_recontext.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/imagen_product_recontext.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/imagen_product_recontext.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/imagen_product_recontext.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>            


| Author |
| --- |
| [Jorj Ismailyan](https://github.com/jismailyan-google) |
| [Katie Nguyen](https://github.com/katiemn) |

## Overview

With Product Recontextualization, you can use Google's cutting-edge Imagen model to generate high-quality images of your products in new scenes and backgrounds.

In this tutorial, you'll learn how to use the Google Gen AI SDK for Python to:
  - Recontextualize multiple product items from locally stored images
  - Recontextualize a product item stored in Cloud Storage


## Get started

### Install Google Gen AI SDK for Python

In [ ]:
%pip install --upgrade --quiet google-genai

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Import libraries

In [3]:
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps
from google import genai
from google.genai.types import (
    Image,
    ProductImage,
    RecontextImageConfig,
    RecontextImageSource,
)
import matplotlib.image as img
import matplotlib.pyplot as plt
import numpy as np

### Set Google Cloud project information and create client

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [4]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Define helper functions to display media

In [5]:
def display_image(
    image,
    max_width: int = 700,
    max_height: int = 400,
) -> None:
    pil_image = typing.cast(PIL_Image.Image, image._pil_image)
    if pil_image.mode != "RGB":
        # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
        pil_image = pil_image.convert("RGB")
    image_width, image_height = pil_image.size
    if max_width < image_width or max_height < image_height:
        # Resize to display a smaller notebook image
        pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
    IPython.display.display(pil_image)


def display_local_image(
    images: list[str],
) -> None:
    fig, axes = plt.subplots(1, len(images), figsize=(12, 6))
    if len(images) == 1:
        axes = np.array([axes])
    for i, ax in enumerate(axes):
        image = img.imread(images[i])
        ax.imshow(image)
        ax.axis("off")
    plt.show()

### Load the image model

In [6]:
product_recontext = "imagen-product-recontext-preview-06-30"

## Product Recontextualization with local files

In this section, you'll download product images and then use them to generate new images with different backgrounds and settings.

Supported Product Categories:
  - **Business & Industrial**
  - **Clothing**
  - **Furniture**
  - **Garden & Yard**
  - **Health & Beauty**
  - **Jewelry**
  - **Shoes**
  - **Sporting goods**
  - **Toys & Games**

### Download product images

If you want to use your own images, you can either change the URLs in the `wget` commands or, if you're using local files, update the`first_product` and `second_product` variables in the next step.

In [ ]:
!wget https://storage.googleapis.com/cloud-samples-data/generative-ai/image/perfume.jpg

!wget https://storage.googleapis.com/cloud-samples-data/generative-ai/image/armchair.png

If you'd like to use different local images, modify the file names below.

In [ ]:
first_product = "perfume.jpg"  # @param {type: 'string'}
second_product = "armchair.png"  # @param {type: 'string'}

display_local_image([first_product, second_product])

### Send the request

In each call, you can specify the following parameters in addition to `product_images`:
 - **Prompt:** Describe the setting you'd like the product to appear in.
 - **Enhance Prompt:** This model offers the option to enhance your provided prompt, which is enabled by default. If you'd like to disable this feature, set `enhance_prompt` to False.
 - **Number of images:** 1 - 4

In [ ]:
prompt = "on a makeup vanity surrounded by a flower vase and string of pearls"  # @param {type: 'string'}

response = client.models.recontext_image(
    model=product_recontext,
    source=RecontextImageSource(
        prompt=prompt,
        product_images=[
            ProductImage(product_image=Image.from_file(location=first_product))
        ],
    ),
    config=RecontextImageConfig(
        enhance_prompt=True,
        output_mime_type="image/jpeg",
        number_of_images=1,
        safety_filter_level="BLOCK_LOW_AND_ABOVE",
    ),
)

display_image(response.generated_images[0].image)

You can also recontextualize products without setting a `prompt` in the request. The model will generate an appropriate background based on the product image.


When generating images you can also set the `safety_filter_level` accordingly:

- `safety_filter_level`
  - `BLOCK_LOW_AND_ABOVE`
  - `BLOCK_MEDIUM_AND_ABOVE`
  - `BLOCK_ONLY_HIGH`
  - `BLOCK_NONE`

In [ ]:
response = client.models.recontext_image(
    model=product_recontext,
    source=RecontextImageSource(
        product_images=[
            ProductImage(product_image=Image.from_file(location=second_product))
        ],
    ),
    config=RecontextImageConfig(
        output_mime_type="image/jpeg",
        number_of_images=1,
        safety_filter_level="BLOCK_LOW_AND_ABOVE",
    ),
)

display_image(response.generated_images[0].image)

## Product Recontextualization with files in Cloud Storage

In this section, you'll use an image of sneakers from Cloud Storage to generate a new product image.

### View sneaker image

The following cell displays the product image from Cloud Storage. You can change the URL to use a different image.

In [ ]:
product_image = PIL_Image.open(
    urllib.request.urlopen(
        "https://storage.googleapis.com/cloud-samples-data/generative-ai/image/shoes.jpg"
    )
)

# Display the image
fig, axis = plt.subplots(1, 2, figsize=(12, 6))
axis[0].imshow(product_image)
for ax in axis:
    ax.axis("off")
plt.show()

### Send the request

A digital watermark ([SynthID](https://deepmind.google/technologies/synthid/)) is added to the images by default. You can specifically enable this by setting the `add_watermark` parameter to `True`. You can also [verify a watermarked image](https://cloud.google.com/vertex-ai/generative-ai/docs/image/verify-watermark) in Vertex AI Studio.

In [ ]:
# fmt: off
prompt = "on a running track"  # @param {type: 'string'}
gcs_uri = "gs://cloud-samples-data/generative-ai/image/shoes.jpg"  # @param {type: 'string'}
# fmt: on


response = client.models.recontext_image(
    model=product_recontext,
    source=RecontextImageSource(
        prompt=prompt,
        product_images=[ProductImage(product_image=Image(gcs_uri=gcs_uri))],
    ),
    config=RecontextImageConfig(
        output_mime_type="image/jpeg",
        enhance_prompt=True,
        number_of_images=1,
        safety_filter_level="BLOCK_LOW_AND_ABOVE",
    ),
)

display_image(response.generated_images[0].image)